# Seismic Risk Assessment - Interactive Visualization

This notebook demonstrates how to use the `seismic-risk` package to assess seismic risk for global aviation infrastructure and visualize the results using [folium](https://python-visualization.github.io/folium/).

## Prerequisites

Install the package with Jupyter dependencies:

```bash
pip install -e ".[jupyter]"
```

## 1. Setup and Configuration

In [ ]:
from seismic_risk.config import SeismicRiskConfig
from seismic_risk.pipeline import run_pipeline
import folium

# Configure the pipeline
config = SeismicRiskConfig(
    min_magnitude=5.0,       # M5.0+ earthquakes
    days_lookback=30,        # Past 30 days
    min_quakes_per_country=3,  # Countries with 3+ qualifying quakes
    max_airport_distance_km=200.0,  # Airports within 200km of a quake
)

print(f"Configuration:")
print(f"  Min magnitude: {config.min_magnitude}")
print(f"  Days lookback: {config.days_lookback}")
print(f"  Min quakes per country: {config.min_quakes_per_country}")
print(f"  Max airport distance: {config.max_airport_distance_km} km")

## 2. Run the Pipeline

This fetches live data from USGS, OurAirports, and REST Countries APIs.

In [ ]:
print("Running seismic risk assessment pipeline...")
print("(This may take a minute on first run due to data downloads)")

results = run_pipeline(config)

print(f"\nFound {len(results)} countries at risk")
print(f"Total exposed airports: {sum(len(r.exposed_airports) for r in results)}")

## 3. View Results Summary

In [ ]:
# Display summary table
for r in results:
    print(f"\n{r.country} ({r.iso_alpha3})")
    print(f"  Risk Score: {r.seismic_hub_risk_score}")
    print(f"  Earthquakes: {r.earthquake_count} (avg M{r.avg_magnitude})")
    print(f"  PAGER Alert: {r.highest_pager_alert or 'none'}")
    print(f"  Exposed Airports: {len(r.exposed_airports)}")
    if r.strongest_earthquake:
        eq = r.strongest_earthquake
        print(f"  Strongest: M{eq.magnitude} on {eq.date} at {eq.depth_km}km depth")

## 4. Create Interactive Map with Folium

Airports are colored by PAGER alert level. Earthquakes are shown as red circles sized by magnitude.

In [ ]:
# Create base map
m = folium.Map(location=[20, 0], zoom_start=2, tiles='OpenStreetMap')

# Color mapping for PAGER alerts
alert_colors = {
    'red': 'red',
    'orange': 'orange',
    'yellow': 'beige',
    'green': 'green',
    None: 'blue',
}

# Add airport markers
for result in results:
    color = alert_colors.get(result.highest_pager_alert, 'blue')
    
    for airport in result.exposed_airports:
        popup_html = f"""
        <b>{airport.name}</b><br>
        IATA: {airport.iata_code}<br>
        Country: {result.country}<br>
        Risk Score: {result.seismic_hub_risk_score}<br>
        Closest Quake: {airport.closest_quake_distance_km} km
        """
        
        folium.Marker(
            location=[airport.latitude, airport.longitude],
            popup=folium.Popup(popup_html, max_width=250),
            tooltip=f"{airport.iata_code} - {airport.name}",
            icon=folium.Icon(color=color, icon='plane', prefix='fa'),
        ).add_to(m)

# Add earthquake circles
for result in results:
    if result.strongest_earthquake:
        eq = result.strongest_earthquake
        popup_html = f"""
        <b>M{eq.magnitude} Earthquake</b><br>
        Date: {eq.date}<br>
        Depth: {eq.depth_km} km<br>
        Country: {result.country}
        """
        
        folium.CircleMarker(
            location=[eq.latitude, eq.longitude],
            radius=eq.magnitude * 3,
            popup=folium.Popup(popup_html, max_width=200),
            tooltip=f"M{eq.magnitude} - {result.country}",
            color='darkred',
            fill=True,
            fillColor='red',
            fillOpacity=0.6,
        ).add_to(m)

print(f"Map created with {sum(len(r.exposed_airports) for r in results)} airports and {len(results)} earthquakes")

## 5. Display the Map

In [ ]:
# Display inline
m

## 6. Export Map to HTML (Optional)

Save the map as a standalone HTML file for sharing.

In [ ]:
# Export to HTML file
output_file = "seismic_risk_map.html"
m.save(output_file)
print(f"Map saved to {output_file}")

## Alternative: Use the CLI

You can also generate maps directly from the command line:

```bash
# Generate HTML map with Leaflet.js
seismic-risk run --format html -o dashboard.html

# Generate GeoJSON for GIS tools
seismic-risk run --format geojson -o risk_data.geojson

# Generate JSON for API consumers
seismic-risk run --format json -o results.json
```